In [66]:
import arcpy
from arcpy.sa import *
import os
import time
arcpy.env.overwriteOutput = True

In [68]:
# load input features and make layer
output_dir = r'J:\lakemapping\postprocess\v5_240410'
auxiliary_dataset_gdb=r'D:\postprocess\v240220\auxiliary_dataset.gdb'
after_mask_gdb=os.path.join(output_dir,f'3_polygon_after_rivermask.gdb')
# region="asia_eu_af"
region='oc'
raw_prediction_fn='lakes_'+region
prefix=region

In [4]:
GLAKES_Res=os.path.join(auxiliary_dataset_gdb,'GLAKES_Res')
arcpy.MakeFeatureLayer_management(GLAKES_Res,'GLAKES_Res')

GLAKES=os.path.join(auxiliary_dataset_gdb,r'D:\lakemapping\0_auxiliary_data\GLAKES\GLAKES\GLAKES.gdb\GLAKES')
arcpy.MakeFeatureLayer_management(GLAKES,'GLAKES')

osm_reservoir=os.path.join(auxiliary_dataset_gdb,'osm_as_natural_water_reservoir_dam')
arcpy.MakeFeatureLayer_management(osm_reservoir,'osm_reservoir')

GeoDAR=r'D:\lakemapping\0_auxiliary_data\GeoDAR\GeoDAR_v10_v11\GeoDAR_v11_reservoirs.shp'
arcpy.MakeFeatureLayer_management(GeoDAR,'GeoDAR')

GLAKES_natural_lake=os.path.join(auxiliary_dataset_gdb,'GLAKES_gte1_natural_lake')
arcpy.MakeFeatureLayer_management(GLAKES_natural_lake,'GLAKES_natural_lake')

<Result 'GLAKES_natural_lake'>

# Function

In [41]:
def selectByLocation(inputFeature,overlapType,selectionFeature,selectionType='NEW_SELECTION'):
    arcpy.SelectLayerByLocation_management(inputFeature,overlapType,selectionFeature,selection_type=selectionType)
    
def selectByAttribute(inputFeature,selectionType='NEW_SELECTION',code=''):
    arcpy.management.SelectLayerByAttribute(inputFeature,selectionType,code)
    
def calculateField(inputFeature,field,code):
    arcpy.CalculateField_management(inputFeature, field,code)

    
def normal_join(inputFeature,joinFeature,outputFeauture,field_list,outLayer):
    print('Prepare the fieldmap for river mask spatial join')
    fieldmappings = arcpy.FieldMappings()
    fieldmappings_all = arcpy.FieldMappings()
    fieldmappings_all.addTable(inputFeature)#未mask
    fieldmappings_all.addTable(joinFeature)
    for field_i in field_list:
        field_idx = fieldmappings_all.findFieldMapIndex(field_i)
        fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))
        
    area_idx = fieldmappings.findFieldMapIndex("area_mask")
    area_fieldmap = fieldmappings.getFieldMap(area_idx)
    area_fieldmap.mergeRule = "sum"
    fieldmappings.replaceFieldMap(area_idx, area_fieldmap)
    
    id_idx = fieldmappings.findFieldMapIndex("ORIG_FID")
    id_fieldmap = fieldmappings.getFieldMap(id_idx)
    id_fieldmap.mergeRule = "range"
    fieldmappings.replaceFieldMap(id_idx, id_fieldmap)
    
    flag_idx = fieldmappings.findFieldMapIndex("flag")
    flag_fieldmap = fieldmappings.getFieldMap(flag_idx)
    flag_fieldmap.mergeRule = "mode"
    fieldmappings.replaceFieldMap(flag_idx, flag_fieldmap)
    
    print('Conduct spatial join')
    arcpy.SpatialJoin_analysis(inputFeature, joinFeature, outputFeauture, "JOIN_ONE_TO_ONE",
                               "KEEP_COMMON", fieldmappings, "SHARE_A_LINE_SEGMENT_WITH")
    lyr=outLayer
    arcpy.MakeFeatureLayer_management(outputFeauture, lyr)
                               
def add_Res_flag(inputFeature):
    selectByAttribute(inputFeature,'NEW_SELECTION', 'flag IS NULL OR "flag" = 0')
    selectByLocation(inputFeature,'INTERSECT',osm_reservoir,'SUBSET_SELECTION')
    arcpy.CalculateField_management(inputFeature, 'flag',  3)

    selectByAttribute(inputFeature,'NEW_SELECTION', 'flag IS NULL OR "flag" = 0')
    selectByLocation(inputFeature,'INTERSECT',GeoDAR,'SUBSET_SELECTION')
    arcpy.CalculateField_management(inputFeature, 'flag',  4)

def calculateOperateByAttribute(inputFeature,code,operate):
    selectByAttribute(inputFeature,'NEW_SELECTION',code)
    arcpy.CalculateField_management(inputFeature, 'operate',  operate)
def calculateOperateByLocation(inputFeature,selectionFeature,operate,selectionType='NEW_SELECTION'):
    selectByLocation(inputFeature,selectionType,selectionFeature)
    arcpy.CalculateField_management(inputFeature, 'operate',operate)
    

# export

In [6]:
# prediction=os.path.join(after_mask_gdb,raw_prediction_fn)
# prediction_lyr='lakes_asia_eu_af'
# arcpy.MakeFeatureLayer_management(prediction,prediction_lyr)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Features: Dataset J:\lakemapping\postprocess\v5_240410\3_polygon_after_rivermask.gdb\lakes_oc does not exist or is not supported
WARNING 000725: Output Layer: Dataset lakes_asia_eu_af already exists.
Failed to execute (MakeFeatureLayer).


In [30]:
prediction=os.path.join(after_mask_gdb,raw_prediction_fn+'_afterMask')
prediction_lyr='lakes_oc'
arcpy.MakeFeatureLayer_management(prediction,prediction_lyr)

<Result 'lakes_oc'>

In [6]:
# GLAKES_clip=os.path.join(after_mask_gdb,'GLAKES_clip_temp')
# GLAKES_clip_split=os.path.join(after_mask_gdb,'GLAKES_clip_split')
# GLAKES_clip_SJ=os.path.join(after_mask_gdb,'GLAKES_clip_SJ')

In [31]:
GLAKES_clip=os.path.join(after_mask_gdb,'GLAKES_oc_clip_temp')
GLAKES_clip_split=os.path.join(after_mask_gdb,'GLAKES_oc_clip_split')
GLAKES_clip_SJ=os.path.join(after_mask_gdb,'GLAKES_oc_clip_SJ')

In [8]:
# arcpy.AddField_management(prediction_lyr, 'flag', "Short")
selectByAttribute(prediction_lyr,'NEW_SELECTION','"flag" IS NULL')
selectByLocation(prediction_lyr,'INTERSECT',GLAKES_Res,"SUBSET_SELECTION")
arcpy.CalculateField_management(prediction_lyr,'flag',2)
selectByAttribute(prediction_lyr,'NEW_SELECTION','"flag" IS NULL')
selectByLocation(prediction_lyr,'INTERSECT',osm_reservoir,"SUBSET_SELECTION")
arcpy.CalculateField_management(prediction_lyr,'flag',3)
selectByAttribute(prediction_lyr,'NEW_SELECTION','"flag" = 3')
selectByLocation(prediction_lyr,'INTERSECT',GeoDAR,"SUBSET_SELECTION")
arcpy.CalculateField_management(prediction_lyr,'flag',4)
selectByAttribute(prediction_lyr,'NEW_SELECTION','')

In [146]:
# arcpy.AddField_management(prediction_lyr, 'area', "Double")
# arcpy.CalculateField_management(prediction_lyr, "area",  "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")

<Result 'lakes_asia_eu'>

In [32]:
selectByAttribute(prediction_lyr,'NEW_SELECTION','"flag" <> 0')

In [33]:
selectByLocation('GLAKES','INTERSECT',prediction_lyr,"NEW_SELECTION")
selectByAttribute('GLAKES','ADD_TO_SELECTION','"Area_bound">=1')

In [34]:
selectByLocation(prediction_lyr,'INTERSECT','GLAKES',"NEW_SELECTION")

In [35]:
# arcpy.analysis.Clip('GLAKES',prediction_lyr,GLAKES_clip)
# arcpy.management.MultipartToSinglepart(GLAKES_clip, GLAKES_clip_split)
# arcpy.MakeFeatureLayer_management(GLAKES_clip_split,'GLAKES_clip_split')

<Result 'GLAKES_clip_split'>

In [36]:
arcpy.analysis.Clip(prediction_lyr,'GLAKES',GLAKES_clip)
arcpy.management.MultipartToSinglepart(GLAKES_clip, GLAKES_clip_split)
arcpy.MakeFeatureLayer_management(GLAKES_clip_split,'GLAKES_clip_split')

<Result 'GLAKES_clip_split'>

In [38]:
print(GLAKES_clip_SJ)

J:\lakemapping\postprocess\v5_240410\3_polygon_after_rivermask.gdb\GLAKES_oc_clip_SJ


In [43]:
# arcpy.AddField_management('GLAKES_oc_clip_split', 'area_mask', "Double")
# arcpy.CalculateField_management('GLAKES_oc_clip_split', "area_mask",  "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")
normal_join('GLAKES','GLAKES_oc_clip_split',GLAKES_clip_SJ,["Area_bound","ORIG_FID","area_mask","Rser_flag",'f'],'GLAKES_oc_clip_SJ')
    #计算比例
arcpy.AddField_management(GLAKES_clip_SJ, 'area_ratio', "DOUBLE")#新建字段为面积比例
arcpy.CalculateField_management(GLAKES_clip_SJ, 'area_ratio', '!area_mask!/!Area_bound!', "PYTHON3")

Prepare the fieldmap for river mask spatial join
Conduct spatial join


<Result 'J:\\lakemapping\\postprocess\\v5_240410\\3_polygon_after_rivermask.gdb\\GLAKES_oc_clip_SJ'>

In [37]:
# gridshp=r'D:\postprocess\GridShp_v240228.shp'
# arcpy.MakeFeatureLayer_management(gridshp,'gridshp')
# arcpy.MakeFeatureLayer_management(GLAKES_clip_SJ,'GLAKES_clip_SJ')
# arcpy.AddField_management('GLAKES_clip_SJ', 'region', "TEXT")
for i in range(2,10):
    selectByAttribute('gridshp','NEW_SELECTION',f"region = 'as_{i}'")
    selectByLocation('GLAKES_clip_SJ','HAVE_THEIR_CENTER_IN','gridshp',"NEW_SELECTION")
    arcpy.CalculateField_management('GLAKES_clip_SJ','region',f"'as_{i}'")
for r in ['eu','af']:
    selectByAttribute('gridshp','NEW_SELECTION',f"region = '{r}'")
    selectByLocation('GLAKES_clip_SJ','HAVE_THEIR_CENTER_IN','gridshp',"NEW_SELECTION")
    arcpy.CalculateField_management('GLAKES_clip_SJ','region',f"'{r}'")

In [93]:
lake_big_lake_clip_temp=lake_big_lake_clip+"_temp"
arcpy.analysis.Clip(result_afm_lyr,lake_big_lake_lyr,lake_big_lake_clip_temp)

<Result 'J:\\lakemapping\\postprocess\\v5_240410\\3_polygon_after_rivermask.gdb\\lakes_eu_biglakeModel_clip_temp'>

In [94]:
arcpy.management.MultipartToSinglepart(lake_big_lake_clip_temp, lake_big_lake_clip)
arcpy.MakeFeatureLayer_management(lake_big_lake_clip,lake_big_lake_clip_lyr)

<Result 'eu_biglakeModel_clip'>

In [95]:
# arcpy.AddField_management(lake_big_lake_clip_lyr, 'area_arm', "Double")
# arcpy.CalculateField_management(lake_big_lake_clip_lyr, "area_arm",  "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")
normal_join(lake_big_lake_lyr,lake_big_lake_clip_lyr,big_lake_join,["area","area_arm","flag","iwR","operate_pre","operate"],big_lake_join_lyr)
    #计算比例
arcpy.AddField_management(big_lake_join_lyr, 'area_ratio_R', "DOUBLE")#新建字段为面积比例
arcpy.CalculateField_management(big_lake_join_lyr, 'area_ratio_R', '!area_arm!/!area!', "PYTHON3")

Prepare the fieldmap for river mask spatial join
Conduct spatial join


<Result 'eu_biglakeModel_SJ'>

In [118]:
arcpy.MakeFeatureLayer_management(big_lake_join,big_lake_join_lyr)

<Result 'as_8_biglakeModel_SJ'>

In [119]:
selectByAttribute(big_lake_join_lyr,'NEW_SELECTION','"operate"=1')
result_afm_merge_temp=result_afm_merge+"_temp"
merge_list=[result_afm_lyr,big_lake_join_lyr]
fieldmappings = arcpy.FieldMappings()
fieldmappings_all = arcpy.FieldMappings()
fieldmappings_all.addTable(result_afm_lyr)
field_list = ['area','iwR','flag','operate_pre']
arcpy.Merge_management(merge_list,result_afm_merge_temp,fieldmappings)
arcpy.Dissolve_management(result_afm_merge_temp,result_afm_merge,multi_part="SINGLE_PART")

<Result 'J:\\lakemapping\\postprocess\\v5_240410\\3_polygon_after_rivermask.gdb\\lakes_eu_afterMask_merge'>

In [120]:
arcpy.MakeFeatureLayer_management(result_afm_merge,result_afm_merge_lyr)
arcpy.AddField_management(result_afm_merge_lyr, 'area', "Double")
arcpy.CalculateField_management(result_afm_merge_lyr, "area",  "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")

<Result 'as_8_afterMask_mege'>